In [1]:
import fitz
import string
import pandas as pd
from difflib import SequenceMatcher

In [2]:
filepath_1to5 = r'REP-EDC-2020_Fusion_Final-1-5.pdf'
filepath_full = r'REP-EDC-2020_Fusion_Final.pdf'

In [3]:
def openPDFasTextDict(filepath):
    """
    Opens PDF as XML dict
    """
    text_dict = []
    with fitz.open(filepath) as doc:
        for page in doc:
            text_dict.append(page.get_text("dict", sort=False))
    return text_dict

In [4]:
def removePuncandSpace(text):
    """
    Removes punctuation and spaces from a string
    """
    return text.translate(str.maketrans('', '', string.punctuation)).strip()

NameError: name 'text_dict_full' is not defined

In [9]:
textDict_1to5 = openPDFasTextDict(filepath = filepath_1to5)
textDict_full = openPDFasTextDict(filepath = filepath_full)

In [12]:
##
# Get an idea of looping through text
# For every page
# Extract additional information about the text as well: font and font size
# Store in list of dictionaries
##

def extractFromTextDict(text_dict_full):

    org_list = []
    org_id = -1
    charitable_foundation = False

    for count_page, page in enumerate(text_dict_full):
        for count_block_list, block_list in enumerate(page["blocks"]):
            for count_line_list, line_list in enumerate(block_list["lines"]):
                for count_spans_list, spans_list in enumerate(line_list["spans"]):
                                    
                    #Remove empty text
                    if spans_list['text'].isspace():
                        continue                
                    
                    #Check if font & size are that of org number or new org
                    if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                        charitable_foundation = False
                        #Check if start of new org
                        try :
                            #Throws ValueError if name of org
                            int(spans_list['text'])
                        except ValueError:
                            #Only triggers when name of org
                            org_list[org_id]['Name'] = spans_list['text'].strip()                   
                        else:
                            #If not name of org then org number
                            if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                                org_number = spans_list['text'].strip()
                                org_list.append({'id' : org_number,
                                                'isFoundation' : 'No'})
                                org_id += 1
                    
                    #Check if not in charitable organisation
                    if not charitable_foundation:
                        
                        #Check if font & size are that of org address
                        #Uses round to filter more text: other text has size that rounds to 8
                        if (spans_list['font'] == 'Helvetica') & (round(float(spans_list['size'])) == 9):
                            #Catch if no orgs created
                            if org_id < 0:
                                continue
                            #If key Address doesn't already exist, create it
                            if 'Address' not in org_list[org_id].keys():
                                org_list[org_id]['Address'] = ''
                                org_list[org_id]['Address'] += spans_list['text']
                            else:
                                #Strip here to avoid unnecessary blank space
                                #Maybe handle this later?
                                org_list[org_id]['Address'] += spans_list['text'].strip()

                        #Check if font & size are that of field name
                        if (spans_list['font'] == 'ArialNarrow') & (int(float(spans_list['size'])) == 8):
                            #Catch if no orgs created
                            if org_id < 0:
                                continue
                            #If key field doesn't already exist, create it. Checks if length string > 1 to remove bad text
                            if (removePuncandSpace(spans_list['text']) not in org_list[org_id].keys()) & (len(removePuncandSpace(spans_list['text'])) > 1):
                                org_list[org_id][removePuncandSpace(spans_list['text'])] = ''

                        #Check if font & size are that of field text
                        if (spans_list['font'] == 'Helvetica-Bold') & (round(float(spans_list['size'])) == 8):
                            #Catch if no orgs created
                            if org_id < 0:
                                continue

                            #Place in last dict key: will always be something there, non-generalizable method
                            org_list[org_id][list( org_list[org_id])[-1]] += spans_list['text']
                    
                    ### Foundations ####
                    #Check if text indicates charitable foundation
                    if spans_list['text'] == "L'entreprise possède une fondation corporative :":
                        charitable_foundation = True
                        org_id += 1
                        org_list.append({'id' : org_number,
                                        'isFoundation' : 'Yes'})
                    
                    #Check if are in charitable foundation
                    if charitable_foundation:
                        #Check if font & size are foundation name
                        if (spans_list['font'] == 'Helvetica-Bold') & (round(float(spans_list['size'])) >= 9):
                            #If key Name doesn't already exist, create it
                            if 'Name' not in org_list[org_id].keys():
                                org_list[org_id]['Name'] = ''
                                org_list[org_id]['Name'] += spans_list['text']
                            else:
                                #Strip here to avoid unnecessary blank space
                                org_list[org_id]['Name'] += spans_list['text'].strip()
                        
                        #Check if font & size are address
                        if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 8):
                            #If key Name doesn't already exist, create it
                            if 'Address' not in org_list[org_id].keys():
                                org_list[org_id]['Address'] = ''
                                org_list[org_id]['Address'] += spans_list['text']
                            else:
                                #Strip here to avoid unnecessary blank space
                                org_list[org_id]['Address'] += ', ' + spans_list['text'].strip()

                
                        #Check if font & size are that of field name
                        if (spans_list['font'] == 'ArialNarrow') & (int(float(spans_list['size'])) == 8):
                            #Catch if no orgs created
                            if org_id < 0:
                                continue
                            #If key field doesn't already exist, create it. Checks if length string > 1 to remove bad text
                            if (removePuncandSpace(spans_list['text']) not in org_list[org_id].keys()) & (len(removePuncandSpace(spans_list['text'])) > 1):
                                org_list[org_id][removePuncandSpace(spans_list['text'])] = ''

                        #Check if font & size are that of field text
                        if (spans_list['font'] == 'ArialNarrow,Bold') & (round(float(spans_list['size'])) == 8):
                            #Catch if no orgs created
                            if org_id < 0:
                                continue

                            #Place in last dict key: will always be something there, non-generalizable method
                            org_list[org_id][list( org_list[org_id])[-1]] += spans_list['text']
    return org_list
                    

In [13]:
extractedList_full = extractFromTextDict(textDict_full)

In [14]:
df_full = pd.DataFrame(extractedList_full)

In [15]:
df_full.head()

,id,isFoundation,Name,Address,Secteur industriel,Langue de comm,DDD,FAF,N° de télCie,n° de tél,...,Date approb,Échelledons,Actif,Princip Filiales,Principale filiale,Filiales princip,Principales filiales,Total actif,Principfiliales,ou
0,1,No,3M Canada inc.,"7290, rue Frederick Banting Saint-Laurent QC ...",Équipement et services industriels; Fabricatio...,Français,En tout temps,31 décembre,514-336-5252,800-265-1840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,No,A & D Prévost,"305, 12e Avenue Richelieu QC J3L3T2","Matériaux de construction, manufacturiers; Fab...",Français,En tout temps,31 décembre,450-658-8771,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,No,AAR Aicraft - Services Trois-Rivières,"3750, chemin de l'aéroport Trois-Rivières QC...","Offre des services d'entretien, de réparation ...",Français,En tout temps,31 décembre,819-377-4500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,No,ABB (Albert Bob Bob),800 boul. Hymes Saint-Laurent QC H4S0B5,Équipement et services industriels; Fabriquer ...,Français,En tout temps,NaN,514-856-6222,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,No,ABB Canada,800 boul. Hymes Saint-Laurent QC H4S0B5,Commerce de gros et détail; Grossistes-distrib...,Français,En tout temps,31 décembre,438-843-6000,888-856-6266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_full[df_full.isFoundation == 'Yes']

,id,isFoundation,Name,Address,Secteur industriel,Langue de comm,DDD,FAF,N° de télCie,n° de tél,...,Date approb,Échelledons,Actif,Princip Filiales,Principale filiale,Filiales princip,Principales filiales,Total actif,Principfiliales,ou
6,6,Yes,"AbbVie,","AbbVie,, Fondation, 8401, rte Trans-Canadienne...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,32,Yes,"Air-Canada,Fondation","Fondation , 7373, boul. de la Côte-Vertu, Sain...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,42,Yes,Alcoa Foundation,"Alcoa Foundation, , The, 390 Park Ave, 9th Flo...",NaN,NaN,NaN,NaN,NaN,NaN,...,En tout temps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,48,Yes,Fondation Memoria,"Fondation Memoria , 1115, rue Laurier O., Outr...",NaN,NaN,NaN,NaN,NaN,NaN,...,En tout temps,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,65,Yes,Allstate Foundation of Canada,"Allstate Foundation of Canada , 27 Allstate Pa...",NaN,NaN,NaN,NaN,NaN,NaN,...,"En tout temps; dates de tombée, octobre et no...",,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,1278,Yes,"Utramar Canada,","Utramar Canada,, La Fondation, 1155, boul. Ren...",NaN,NaN,NaN,NaN,NaN,NaN,...,En tout temps,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1352,1282,Yes,NaN,La Fondation Unilever a été révoquée volontair...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1357,1286,Yes,"UPS Foundation,","UPS Foundation,, The, c/o UPS Canada Ltd, 6285...",NaN,NaN,NaN,NaN,NaN,NaN,...,En tout temps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1374,1302,Yes,"Velan Foundation,","Velan Foundation,, Fondation, 7007, ch. Côte-d...",NaN,NaN,NaN,NaN,NaN,NaN,...,En tout temps,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_full.to_excel('DF_v1.xlsx', index=None)

In [25]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [28]:
#Loop through dataframe column pairs
for col1 in df_full.columns:
    for col2 in df_full.columns:
        
        if col1 == col2:
            continue
        
        if similar(col1, col2) >= 0.8:
            print(col1 + " - " + col2 + ": " + str(similar(col1,col2)))

n° de tél - 2e n° de tél: 0.8571428571428571
Domaine dintérêt - Domaines dintérêt: 0.9696969696969697
2e n° de tél - n° de tél: 0.8571428571428571
Domaines dintérêt - Domaine dintérêt: 0.9696969696969697
Princip filiales - Princip Filiales: 0.9375
Princip filiales - Principale filiale: 0.8823529411764706
Princip filiales - Principales filiales: 0.8888888888888888
Princip filiales - Principfiliales: 0.967741935483871
Princip Filiales - Princip filiales: 0.9375
Princip Filiales - Principale filiale: 0.8235294117647058
Princip Filiales - Principales filiales: 0.8333333333333334
Princip Filiales - Principfiliales: 0.9032258064516129
Principale filiale - Princip filiales: 0.8823529411764706
Principale filiale - Princip Filiales: 0.8235294117647058
Principale filiale - Principales filiales: 0.9473684210526315
Principale filiale - Principfiliales: 0.8484848484848485
Principales filiales - Princip filiales: 0.8888888888888888
Principales filiales - Princip Filiales: 0.8333333333333334
Principa